In [3]:
## Retrieval agumented generation

In [1]:
import os
from dotenv import load_dotenv
import streamlit as st
from llama_index.core.service_context import ServiceContext
# from llama_index.llms import OpenAI
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.graph_stores.nebula import NebulaGraphStore
from llama_index.core.llms import LLM
from llama_index.llms.openai import OpenAI
from llama_index.core.response.pprint_utils import pprint_response

load_dotenv()

True

In [4]:
os.environ['OPENAI_API_KEY']= os.getenv('OPEN_API_KEY')
documents= SimpleDirectoryReader("data").load_data()

In [5]:
index= VectorStoreIndex.from_documents(documents, show_progress=2)

Generating embeddings: 100%|██████████| 99/99 [00:02<00:00, 46.08it/s]


In [10]:
index

In [11]:
query_engin= index.as_query_engine()

In [32]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor
from llama_index.core.data_structs import Node
from llama_index.core.schema import NodeWithScore

# Set similarity_top_k=4 to retrieve 4 similar source from where it get the answere 
retriever = VectorIndexRetriever(index= index, similarity_top_k=4)
#Set the similartiy to specfic threshold
nodes = [
    NodeWithScore(node=Node(text="text1"), score=0.7),
    NodeWithScore(node=Node(text="text2"), score=0.8),
]
postprocessor= SimilarityPostprocessor(similarity_cutoff= 0.80)
query_engin= RetrieverQueryEngine(retriever=retriever, node_postprocessors= [postprocessor])

In [33]:
resopnse= query_engin.query("What is attention is all you need?")
# Adding show_source= Ture to give from where it get the answer and with how much silimarty
pprint_response(resopnse, show_source= True)

Final Response: Attention is the process that enables us to select
important information for further processing while inhibiting other
information. It helps us focus on specific stimuli in our environment,
allowing us to navigate through the overwhelming amount of information
we encounter. By choosing what to pay attention to, we can effectively
carry out tasks and respond to relevant stimuli.
______________________________________________________________________
Source Node 1/4
Node ID: 009d0bed-88ec-4f6b-890a-42cb5c5a07f7
Similarity: 0.8217520449932787
Text: 104CHAPTER 3 Attentionwe manage to keep from being overloaded
and thus rendered incapable of action? How dowe, moment to moment,
choose the information that is meaningful and avoid distraction
byirrelevant material? One solution is to focus on some particular
piece of information (such asthe sound of your own name or a color of
interest) and to ...
______________________________________________________________________
Source Node

In [35]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What are transformers?")
print(response)

Transformers are a type of mechanism integrated into architectures to extract hierarchical features from data. They typically involve self-attention mechanisms and are used to improve the learning process for positive target instances while reducing the rate of missed detections.
